In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [2]:
with open('../datakeep/plant64/train_64_nparray.atikeep', 'rb') as handle:
    X,y = pickle.load(handle)

In [3]:
#load everything
import torch
from torchvision import datasets, transforms
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.utils import make_grid

import sys, time
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [4]:
from sklearn import preprocessing

In [5]:
le = preprocessing.LabelEncoder()

In [6]:
le.fit(y)

LabelEncoder()

In [7]:
len(le.classes_)

39

In [8]:
y_lebeled = le.transform(y)

In [9]:
y_lebeled[:5]

array([29, 26,  4, 17, 12])

In [10]:
convert_tensor = transforms.ToTensor()

In [11]:
X_train = []
for eX in X:
    X_train.append(convert_tensor(eX))
X = torch.stack(X_train)

In [13]:
#please create tensors from these numpy array
#torch.from_numpy (copy)  or torch.tensor  (not a copy!)
# inputs  = torch.tensor(X)
targets = torch.tensor(y_lebeled)
inputs = X
#please print the shape of these tensors
#use either .size() or .shape
inputs.shape, targets.shape


(torch.Size([43040, 3, 64, 64]), torch.Size([43040]))

In [14]:
from torch.utils.data import TensorDataset


In [15]:
ds = TensorDataset(inputs, targets)

In [16]:
X.shape

torch.Size([43040, 3, 64, 64])

In [17]:
43040 - 8600

34440

In [18]:
train_set, val_set = torch.utils.data.random_split(ds, [34440, 8600])

In [20]:
class ConvolutionalNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 3, 1)
        self.conv2 = nn.Conv2d(6, 16, 3, 1)
        self.fc1 = nn.Linear(16*14 * 14, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 39)

    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)
        # print(X.shape)
        X = X.view(-1, 16*14 * 14)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)
        return X

In [21]:
torch.manual_seed(101)
CNNmodel  = ConvolutionalNetwork()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(CNNmodel.parameters(), lr=0.001)

In [22]:
def count_parameters(model):
    params = [p.numel() for p in model.parameters() if p.requires_grad]
    for item in params:
        print(f'{item:>8}')
    print(f'________\n{sum(params):>8}')

In [23]:
#create Dataloader
torch.manual_seed(9999) #you can put this in the top....

batch_size = 32
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

In [24]:
import torch.nn.functional as F

In [25]:
import time
start_time = time.time()

epochs = 3

max_trn_batch = 800
max_tst_batch = 300

train_losses = []
test_losses = []
train_correct = []
test_correct = []

for i in range(epochs):
    trn_corr = 0
    tst_corr = 0
    
    # Run the training batches
    for b, (X_train, y_train) in enumerate(train_loader):
        
        # Limit the number of batches
        if b == max_trn_batch:
            break
        b+=1
        
        # Apply the model
        y_pred = CNNmodel(X_train)
        loss = criterion(y_pred, y_train)
 
        # Tally the number of correct predictions
        predicted = torch.max(y_pred.data, 1)[1]
        batch_corr = (predicted == y_train).sum()
        trn_corr += batch_corr
        
        # Update parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print interim results
        if b%200 == 0:
            print(f'epoch: {i:2}  batch: {b:4} [{10*b:6}/8000]  loss: {loss.item():10.8f}  \
accuracy: {trn_corr.item()*100/(10*b):7.3f}%')

epoch:  0  batch:  200 [  2000/8000]  loss: 2.26036525  accuracy:  50.800%
epoch:  0  batch:  400 [  4000/8000]  loss: 1.79283273  accuracy:  85.775%
epoch:  0  batch:  600 [  6000/8000]  loss: 1.78383374  accuracy: 108.717%
epoch:  0  batch:  800 [  8000/8000]  loss: 1.43850577  accuracy: 125.425%
epoch:  1  batch:  200 [  2000/8000]  loss: 1.02816772  accuracy: 193.050%
epoch:  1  batch:  400 [  4000/8000]  loss: 1.10624051  accuracy: 196.750%
epoch:  1  batch:  600 [  6000/8000]  loss: 1.17746508  accuracy: 199.333%
epoch:  1  batch:  800 [  8000/8000]  loss: 1.10609531  accuracy: 201.850%
epoch:  2  batch:  200 [  2000/8000]  loss: 1.17281723  accuracy: 218.150%
epoch:  2  batch:  400 [  4000/8000]  loss: 1.16349530  accuracy: 218.125%
epoch:  2  batch:  600 [  6000/8000]  loss: 0.69181627  accuracy: 220.250%
epoch:  2  batch:  800 [  8000/8000]  loss: 0.80879033  accuracy: 222.125%
